<a href="https://colab.research.google.com/github/Imurdie/Automatic_Model_Compression/blob/main/AMC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sun Mar  5 14:13:06 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P0    28W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%cd '/content/amc'

/content/amc


In [ ]:
!git clone https://github.com/mit-han-lab/amc

Cloning into 'amc'...
remote: Enumerating objects: 73, done.
remote: Total 73 (delta 0), reused 0 (delta 0), pack-reused 73
Unpacking objects: 100% (73/73), done.


In [2]:
!pwd

/content


##too old to run

In [ ]:
# Code for "AMC: AutoML for Model Compression and Acceleration on Mobile Devices"
# Yihui He*, Ji Lin*, Zhijian Liu, Hanrui Wang, Li-Jia Li, Song Han
# {jilin, songhan}@mit.edu

import os
import numpy as np
import argparse
from copy import deepcopy
import torch
torch.backends.cudnn.deterministic = True

from env.channel_pruning_env import ChannelPruningEnv
from lib.agent import DDPG
from lib.utils import get_output_folder

from tensorboardX import SummaryWriter


def parse_args():
    parser = argparse.ArgumentParser(description='AMC search script')

    parser.add_argument('--job', default='train', type=str, help='support option: train/export')
    parser.add_argument('--suffix', default=None, type=str, help='suffix to help you remember what experiment you ran')
    # env
    parser.add_argument('--model', default='mobilenet', type=str, help='model to prune')
    parser.add_argument('--dataset', default='imagenet', type=str, help='dataset to use (cifar/imagenet)')
    parser.add_argument('--data_root', default=None, type=str, help='dataset path')
    parser.add_argument('--preserve_ratio', default=0.5, type=float, help='preserve ratio of the model')
    parser.add_argument('--lbound', default=0.2, type=float, help='minimum preserve ratio')
    parser.add_argument('--rbound', default=1., type=float, help='maximum preserve ratio')
    parser.add_argument('--reward', default='acc_reward', type=str, help='Setting the reward')
    parser.add_argument('--acc_metric', default='acc5', type=str, help='use acc1 or acc5')
    parser.add_argument('--use_real_val', dest='use_real_val', action='store_true')
    parser.add_argument('--ckpt_path', default=None, type=str, help='manual path of checkpoint')
    # parser.add_argument('--pruning_method', default='cp', type=str,
    #                     help='method to prune (fg/cp for fine-grained and channel pruning)')
    # only for channel pruning
    parser.add_argument('--n_calibration_batches', default=60, type=int,
                        help='n_calibration_batches')
    parser.add_argument('--n_points_per_layer', default=10, type=int,
                        help='method to prune (fg/cp for fine-grained and channel pruning)')
    parser.add_argument('--channel_round', default=8, type=int, help='Round channel to multiple of channel_round')
    # ddpg
    parser.add_argument('--hidden1', default=300, type=int, help='hidden num of first fully connect layer')
    parser.add_argument('--hidden2', default=300, type=int, help='hidden num of second fully connect layer')
    parser.add_argument('--lr_c', default=1e-3, type=float, help='learning rate for actor')
    parser.add_argument('--lr_a', default=1e-4, type=float, help='learning rate for actor')
    parser.add_argument('--warmup', default=100, type=int,
                        help='time without training but only filling the replay memory')
    parser.add_argument('--discount', default=1., type=float, help='')
    parser.add_argument('--bsize', default=64, type=int, help='minibatch size')
    parser.add_argument('--rmsize', default=100, type=int, help='memory size for each layer')
    parser.add_argument('--window_length', default=1, type=int, help='')
    parser.add_argument('--tau', default=0.01, type=float, help='moving average for target network')
    # noise (truncated normal distribution)
    parser.add_argument('--init_delta', default=0.5, type=float,
                        help='initial variance of truncated normal distribution')
    parser.add_argument('--delta_decay', default=0.95, type=float,
                        help='delta decay during exploration')
    # training
    parser.add_argument('--max_episode_length', default=1e9, type=int, help='')
    parser.add_argument('--output', default='./logs', type=str, help='')
    parser.add_argument('--debug', dest='debug', action='store_true')
    parser.add_argument('--init_w', default=0.003, type=float, help='')
    parser.add_argument('--train_episode', default=800, type=int, help='train iters each timestep')
    parser.add_argument('--epsilon', default=50000, type=int, help='linear decay of exploration policy')
    parser.add_argument('--seed', default=None, type=int, help='random seed to set')
    parser.add_argument('--n_gpu', default=1, type=int, help='number of gpu to use')
    parser.add_argument('--n_worker', default=16, type=int, help='number of data loader worker')
    parser.add_argument('--data_bsize', default=50, type=int, help='number of data batch size')
    parser.add_argument('--resume', default='default', type=str, help='Resuming model path for testing')
    # export
    parser.add_argument('--ratios', default=None, type=str, help='ratios for pruning')
    parser.add_argument('--channels', default=None, type=str, help='channels after pruning')
    parser.add_argument('--export_path', default=None, type=str, help='path for exporting models')
    parser.add_argument('--use_new_input', dest='use_new_input', action='store_true', help='use new input feature')

    return parser.parse_args()


def get_model_and_checkpoint(model, dataset, checkpoint_path, n_gpu=1):
    if model == 'mobilenet' and dataset == 'imagenet':
        from models.mobilenet import MobileNet
        net = MobileNet(n_class=1000)
    elif model == 'mobilenetv2' and dataset == 'imagenet':
        from models.mobilenet_v2 import MobileNetV2
        net = MobileNetV2(n_class=1000)
    else:
        raise NotImplementedError
    sd = torch.load(checkpoint_path)
    if 'state_dict' in sd:  # a checkpoint but not a state_dict
        sd = sd['state_dict']
    sd = {k.replace('module.', ''): v for k, v in sd.items()}
    net.load_state_dict(sd)
    net = net.cuda()
    if n_gpu > 1:
        net = torch.nn.DataParallel(net, range(n_gpu))

    return net, deepcopy(net.state_dict())


def train(num_episode, agent, env, output):
    agent.is_training = True
    step = episode = episode_steps = 0
    episode_reward = 0.
    observation = None
    T = []  # trajectory
    while episode < num_episode:  # counting based on episode
        # reset if it is the start of episode
        if observation is None:
            observation = deepcopy(env.reset())
            agent.reset(observation)

        # agent pick action ...
        if episode <= args.warmup:
            action = agent.random_action()
            # action = sample_from_truncated_normal_distribution(lower=0., upper=1., mu=env.preserve_ratio, sigma=0.5)
        else:
            action = agent.select_action(observation, episode=episode)

        # env response with next_observation, reward, terminate_info
        observation2, reward, done, info = env.step(action)
        observation2 = deepcopy(observation2)

        T.append([reward, deepcopy(observation), deepcopy(observation2), action, done])

        # fix-length, never reach here
        # if max_episode_length and episode_steps >= max_episode_length - 1:
        #     done = True

        # [optional] save intermideate model
        if episode % int(num_episode / 3) == 0:
            agent.save_model(output)

        # update
        step += 1
        episode_steps += 1
        episode_reward += reward
        observation = deepcopy(observation2)

        if done:  # end of episode
            print('#{}: episode_reward:{:.4f} acc: {:.4f}, ratio: {:.4f}'.format(episode, episode_reward,
                                                                                 info['accuracy'],
                                                                                 info['compress_ratio']))
            text_writer.write(
                '#{}: episode_reward:{:.4f} acc: {:.4f}, ratio: {:.4f}\n'.format(episode, episode_reward,
                                                                                 info['accuracy'],
                                                                                 info['compress_ratio']))
            final_reward = T[-1][0]
            # print('final_reward: {}'.format(final_reward))
            # agent observe and update policy
            for r_t, s_t, s_t1, a_t, done in T:
                agent.observe(final_reward, s_t, s_t1, a_t, done)
                if episode > args.warmup:
                    agent.update_policy()

            #agent.memory.append(
            #    observation,
            #    agent.select_action(observation, episode=episode),
            #    0., False
            #)

            # reset
            observation = None
            episode_steps = 0
            episode_reward = 0.
            episode += 1
            T = []

            tfwriter.add_scalar('reward/last', final_reward, episode)
            tfwriter.add_scalar('reward/best', env.best_reward, episode)
            tfwriter.add_scalar('info/accuracy', info['accuracy'], episode)
            tfwriter.add_scalar('info/compress_ratio', info['compress_ratio'], episode)
            tfwriter.add_text('info/best_policy', str(env.best_strategy), episode)
            # record the preserve rate for each layer
            for i, preserve_rate in enumerate(env.strategy):
                tfwriter.add_scalar('preserve_rate/{}'.format(i), preserve_rate, episode)

            text_writer.write('best reward: {}\n'.format(env.best_reward))
            text_writer.write('best policy: {}\n'.format(env.best_strategy))
    text_writer.close()


def export_model(env, args):
    assert args.ratios is not None or args.channels is not None, 'Please provide a valid ratio list or pruned channels'
    assert args.export_path is not None, 'Please provide a valid export path'
    env.set_export_path(args.export_path)

    print('=> Original model channels: {}'.format(env.org_channels))
    if args.ratios:
        ratios = args.ratios.split(',')
        ratios = [float(r) for r in ratios]
        assert  len(ratios) == len(env.org_channels)
        channels = [int(r * c) for r, c in zip(ratios, env.org_channels)]
    else:
        channels = args.channels.split(',')
        channels = [int(r) for r in channels]
        ratios = [c2 / c1 for c2, c1 in zip(channels, env.org_channels)]
    print('=> Pruning with ratios: {}'.format(ratios))
    print('=> Channels after pruning: {}'.format(channels))

    for r in ratios:
        env.step(r)

    return


if __name__ == "__main__":
    args = parse_args()

    if args.seed is not None:
        np.random.seed(args.seed)
        torch.manual_seed(args.seed)
        torch.cuda.manual_seed(args.seed)

    model, checkpoint = get_model_and_checkpoint(args.model, args.dataset, checkpoint_path=args.ckpt_path,
                                                 n_gpu=args.n_gpu)

    env = ChannelPruningEnv(model, checkpoint, args.dataset,
                            preserve_ratio=1. if args.job == 'export' else args.preserve_ratio,
                            n_data_worker=args.n_worker, batch_size=args.data_bsize,
                            args=args, export_model=args.job == 'export', use_new_input=args.use_new_input)

    if args.job == 'train':
        # build folder and logs
        base_folder_name = '{}_{}_r{}_search'.format(args.model, args.dataset, args.preserve_ratio)
        if args.suffix is not None:
            base_folder_name = base_folder_name + '_' + args.suffix
        args.output = get_output_folder(args.output, base_folder_name)
        print('=> Saving logs to {}'.format(args.output))
        tfwriter = SummaryWriter(logdir=args.output)
        text_writer = open(os.path.join(args.output, 'log.txt'), 'w')
        print('=> Output path: {}...'.format(args.output))

        nb_states = env.layer_embedding.shape[1]
        nb_actions = 1  # just 1 action here

        args.rmsize = args.rmsize * len(env.prunable_idx)  # for each layer
        print('** Actual replay buffer size: {}'.format(args.rmsize))

        agent = DDPG(nb_states, nb_actions, args)
        train(args.train_episode, agent, env, args.output)
    elif args.job == 'export':
        export_model(env, args)
    else:
        raise RuntimeError('Undefined job {}'.format(args.job))


ValueError: ignored

##torch-pruing

In [1]:
!git clone https://github.com/0601p/Model-Compression.git

Cloning into 'Model-Compression'...
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 41 (delta 14), reused 18 (delta 7), pack-reused 16
Unpacking objects: 100% (41/41), 105.87 MiB | 10.38 MiB/s, done.


In [8]:
!pip install torch-pruning # v1.0.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
import os
train_filePath = '/content/12kDriveEnd_img0.2/train'
train_file_path_list_ori = os.listdir(train_filePath)
val_filePath = '/content/12kDriveEnd_img0.2/val'
val_file_path_list_ori = os.listdir(val_filePath)
test_filePath = '/content/12kDriveEnd_img0.2/test'
test_file_path_list_ori = os.listdir(test_filePath)

# file_path_list_ori[:5]

In [10]:
train_file_name_list = list(set([file_path.split('-')[0] for file_path in train_file_path_list_ori]))
val_file_name_list = list(set([file_path.split('-')[0] for file_path in val_file_path_list_ori]))
test_file_name_list = list(set([file_path.split('-')[0] for file_path in test_file_path_list_ori]))
# file_name_list[:5],len(file_name_list)
# file_name_list,len(file_name_list)

In [11]:
def get_label_num(file_name):
  if 'B007' in file_name:
    return 3
  elif 'B014' in file_name:
    return 6
  elif 'B021' in file_name:
    return 9
  elif 'OR007' in file_name:
    return 1
  elif 'OR014' in file_name:
    return 4
  elif 'OR021' in file_name:
    return 7
  elif 'IR007' in file_name:
    return 2
  elif 'IR014' in file_name:
    return 5
  elif 'IR021' in file_name:
    return 8
  else :
    return 0
get_label_num('OR1007@12_3_60568')

0

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

In [13]:
import os
import pandas as pd
from torchvision.io import read_image
from PIL import Image

train_img_dir = '/content/12kDriveEnd_img0.2/train'
val_img_dir = '/content/12kDriveEnd_img0.2/val'
test_img_dir = '/content/12kDriveEnd_img0.2/test'
class CustomImageDataset(Dataset):
    # def __init__(self, file_name_list, img_dir,img_transform):
    def __init__(self, file_name_list, img_dir):
        self.file_name_list = file_name_list
        self.img_dir = img_dir
        # self.img_transform = img_transform

    def __len__(self):
        return len(self.file_name_list)

    def __getitem__(self, idx):
        file_name = self.file_name_list[idx]
        img_path_m = os.path.join(self.img_dir, self.file_name_list[idx]+'-M.png')
        img_path_g = os.path.join(self.img_dir, self.file_name_list[idx]+'-G.png')
        image_m = read_image(img_path_m)
        image_g = read_image(img_path_m)
        # image_m = Image.open(img_path_m)
        # image_g = Image.open(img_path_g)
        # image_tensor_m = self.img_transform(image_m)
        # image_tensor_g = self.img_transform(image_g)
        # label_name = file_name.split('_')[0]
        # label_id = le.transform([label_name])[0]
        label_id = get_label_num(file_name)

        return (image_m,image_g), label_id

# train_img_dataset = CustomImageDataset( file_name_list, train_img_dir,img_transform)
# val_img_dataset = CustomImageDataset( file_name_list, val_img_dir,img_transform)
# test_img_dataset = CustomImageDataset( file_name_list, test_img_dir,img_transform)
train_img_dataset = CustomImageDataset( train_file_name_list, train_img_dir)
val_img_dataset = CustomImageDataset( val_file_name_list, val_img_dir)
test_img_dataset = CustomImageDataset( test_file_name_list, test_img_dir)
train_img_dataset[2]

((tensor([[[42, 40, 27,  ..., 44, 39, 34],
           [42, 40, 26,  ..., 43, 39, 34],
           [27, 29, 36,  ..., 26, 29, 32],
           ...,
           [42, 39, 28,  ..., 44, 38, 33],
           [39, 38, 29,  ..., 39, 37, 33],
           [31, 32, 35,  ..., 31, 32, 33]]], dtype=torch.uint8),
  tensor([[[42, 40, 27,  ..., 44, 39, 34],
           [42, 40, 26,  ..., 43, 39, 34],
           [27, 29, 36,  ..., 26, 29, 32],
           ...,
           [42, 39, 28,  ..., 44, 38, 33],
           [39, 38, 29,  ..., 39, 37, 33],
           [31, 32, 35,  ..., 31, 32, 33]]], dtype=torch.uint8)),
 6)

In [14]:
train_dataloader = DataLoader(train_img_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_img_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_img_dataset, batch_size=64, shuffle=True)

In [15]:

import torch
import torch.nn.utils.prune as prune
import torchvision
import random
import sys
import torch_pruning
# sys.path.append("/content/Model-Compression")

# from model import *

device = 'cuda' if torch.cuda.is_available() else 'cpu'
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed(777)

In [4]:
learning_rate = 1e-4
batch_size = 512
epoch = 20
     

In [5]:
root = "./CIFAR100"
transform = torchvision.transforms.ToTensor()
data_train = torchvision.datasets.CIFAR100(root, train = True, transform = transform, download = True)
data_test = torchvision.datasets.CIFAR100(root, train = False, transform = transform, download = True)

  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting ./CIFAR100/cifar-100-python.tar.gz to ./CIFAR100
Files already downloaded and verified


In [6]:

train_loader = torch.utils.data.DataLoader(dataset=data_train, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = torch.utils.data.DataLoader(dataset=data_test, batch_size=batch_size, shuffle=False, drop_last=True)

In [ ]:
model = final_model_2.to(device)
checkpoint = torch.load("/content/Model-Compression/model.pth")
model.load_state_dict(checkpoint)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [7]:
# model = ResNet(Bottleneck, [3, 4, 6, 3], num_classes=100).to(device)
# checkpoint = torch.load("/content/Model-Compression/model.pth")
# model.load_state_dict(checkpoint)
# criterion = torch.nn.CrossEntropyLoss().to(device)
# optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
def train_one_epoch(print_result = False):
    model.train()
    running_loss = 0.0
    accuracy_sum = 0.0
    length = 0

    for i, data in enumerate(train_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        (img_m,img_g), labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = final_model_1(img_m,img_g)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 10 == 9:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 10:.3f}')
            running_loss = 0.0
    if(print_result):
        print("loss :", loss_sum / length)
        print("accuracy:", accuracy_sum / length)



In [8]:
def train_one_epoch(print_result = False):
    model.train()
    loss_sum = 0.0
    accuracy_sum = 0.0
    length = 0

    for X, Y in train_loader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()

        pred = model(X)
        loss = criterion(pred, Y)
        pred_idx = torch.argmax(pred, 1)
        loss_sum += loss.item()
        accuracy_sum += torch.sum((pred_idx == Y).float()).item()
        length += X.size(0)

        loss.backward()
        optimizer.step()

    if(print_result):
        print("loss :", loss_sum / length)
        print("accuracy:", accuracy_sum / length)

In [ ]:
def eval():
  with torch.no_grad():
      model.eval()
      running_loss = 0.0
      accuracy_sum = 0.0
      length = 0  
      for i, data in enumerate(train_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        (img_m,img_g), labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = final_model_1(img_m,img_g)
        loss = criterion(outputs, labels)

        # print statistics
        running_loss += loss.item()
        if i % 10 == 9:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 10:.3f}')
            running_loss = 0.0


In [9]:
# def eval():
#     with torch.no_grad():
#         model.eval()
#         loss_sum = 0.0
#         accuracy_sum = 0.0
#         length = 0

#         for X, Y in train_loader:
#             X = X.to(device)
#             Y = Y.to(device)

#             pred = model(X)
#             loss = criterion(pred, Y)
#             pred_idx = torch.argmax(pred, 1)
#             loss_sum += loss.item()
#             accuracy_sum += torch.sum((pred_idx == Y).float()).item()
#             length += X.size(0)
            
#         print("loss :", loss_sum / length)
#         print("accuracy:", accuracy_sum / length)

In [ ]:
def train():
    for i in range(epoch):
        print("EPOCH[" + str(i + 1) + "]")
        print("==== train ====")
        train_one_epoch(print_result=True)
        
        print("==== eval ====")
        eval()

In [10]:
# def train():
#     for i in range(epoch):
#         print("EPOCH[" + str(i + 1) + "]")
#         print("==== train ====")
#         train_one_epoch(print_result=True)
        
#         print("==== eval ====")
#         eval()
     

In [ ]:
# code for pruning whole model

example_inputs = torch.randn(64, 1, 200, 200).to("cuda")
imp = torch_pruning.importance.MagnitudeImportance(p = 2)
ignored_layers = []
for m in model.modules():
    if isinstance(m, torch.nn.Linear) and m.out_features == 100:
        ignored_layers.append(m)

#ch_sparsity = 0.5 means 50% of the filters are pruned
pruner = torch_pruning.pruner.MagnitudePruner(model=model, example_inputs=example_inputs, importance=imp, iterative_steps=5, ch_sparsity=0.7, ignored_layers=ignored_layers)

base_macs, base_nparams = torch_pruning.utils.count_ops_and_params(model=model, example_inputs=example_inputs)
print("basemacs :", base_macs, "basenparams :", base_nparams)
for i in range(5):
    pruner.step()
    macs, nparams = torch_pruning.utils.count_ops_and_params(model=model, example_inputs=example_inputs)
    print("iter [", i, "] macs :", macs, "nparams :", nparams)

    #finetune
    optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

    print("==== train ====")
    for j in range(10):
        train_one_epoch(print_result = True)
    
    print("==== eval ====")
    eval()

In [11]:
# # code for pruning whole model

# example_inputs = torch.randn(1, 3, 32, 32).to("cuda")
# imp = torch_pruning.importance.MagnitudeImportance(p = 2)
# ignored_layers = []
# for m in model.modules():
#     if isinstance(m, torch.nn.Linear) and m.out_features == 100:
#         ignored_layers.append(m)

# #ch_sparsity = 0.5 means 50% of the filters are pruned
# pruner = torch_pruning.pruner.MagnitudePruner(model=model, example_inputs=example_inputs, importance=imp, iterative_steps=5, ch_sparsity=0.7, ignored_layers=ignored_layers)

# base_macs, base_nparams = torch_pruning.utils.count_ops_and_params(model=model, example_inputs=example_inputs)
# print("basemacs :", base_macs, "basenparams :", base_nparams)
# for i in range(5):
#     pruner.step()
#     macs, nparams = torch_pruning.utils.count_ops_and_params(model=model, example_inputs=example_inputs)
#     print("iter [", i, "] macs :", macs, "nparams :", nparams)

#     #finetune
#     optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

#     print("==== train ====")
#     for j in range(10):
#         train_one_epoch(print_result = True)
    
#     print("==== eval ====")
#     eval()

basemacs : 84284004.0 basenparams : 23712932
iter [ 0 ] macs : 62645652.0 nparams : 17548123
==== train ====
loss : 0.000151031017069436
accuracy: 0.975354381443299
loss : 9.503110687574852e-05
accuracy: 0.9847575708762887
loss : 6.685144127032734e-05
accuracy: 0.9900531572164949
loss : 5.007407867431295e-05
accuracy: 0.9928519652061856
loss : 4.1293786313131536e-05
accuracy: 0.9951272551546392
loss : 3.213056150287361e-05
accuracy: 0.9965769974226805
loss : 2.7985589815553915e-05
accuracy: 0.9974629510309279
loss : 2.3689339952360353e-05
accuracy: 0.9982482280927835
loss : 2.129067128332759e-05
accuracy: 0.9985502577319587
loss : 1.859226203939131e-05
accuracy: 0.9987717461340206
==== eval ====
loss : 1.0184744374764142e-05
accuracy: 0.999597293814433
iter [ 1 ] macs : 44222598.0 nparams : 12313842
==== train ====
loss : 0.0013549754914549208
accuracy: 0.827722293814433
loss : 0.0010884487515401824
accuracy: 0.8513611469072165
loss : 0.0009099636448921694
accuracy: 0.870227931701031
l

In [12]:
eval()

loss : 0.009169269487569012
accuracy: 0.19418492268041238


#Faulting signals transformed into imgs

##GAF preprocessing with Equal difference 0.2
改成后缀-G,并和-M放到一个文件夹

In [2]:
!pip install pyts

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.3 MB/s eta 0:00:00


In [3]:
!git clone https://github.com/zzwalala/Motor_Bearing_Fault_Diagnosis

Cloning into 'Motor_Bearing_Fault_Diagnosis'...
remote: Enumerating objects: 116, done.
remote: Total 116 (delta 0), reused 0 (delta 0), pack-reused 116
Receiving objects: 100% (116/116), 65.34 MiB | 18.80 MiB/s, done.
Resolving deltas: 100% (16/16), done.


In [4]:
import scipy.io
import random
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import image
from pyts.image import GramianAngularField
import time
import math
# def folder2img(save_path, folder_path, train_num, val_num, test_num, img_size):
def folder2img(save_path, folder_path, train_num, val_num, test_num):
    files = os.listdir(folder_path)
    print(f'\n {len(files)}')
    if not os.path.exists(save_path + '/train'):
        os.makedirs(save_path + '/train')
    if not os.path.exists(save_path + '/val'):
        os.makedirs(save_path + '/val')
    if not os.path.exists(save_path + '/test'):
        os.makedirs(save_path + '/test')
    for file in files:
        
     if os.path.splitext(file)[1] == '.mat':
          file_path = folder_path + "/" + file
          raw_data = scipy.io.loadmat(file_path)
          for name in raw_data.keys():
              if "DE" in name:
                  column_DE = name
                  print(f'\n{column_DE}')
          data_DE = raw_data[column_DE]
          
          n_sample,_, = data_DE.shape
          print(n_sample)
          final_data=data_DE
            
          # im_size = 128

          series = 1000   # each time series

          # im_size = 200
          # step = 1000  # step of slide window
          
          ss = n_sample - series
          # im_sum = (n_sample - im_size) // step
          gasf = GramianAngularField(image_size=0.2, method='difference')
          if ("_3" in os.path.splitext(file)[0]) or ("_0" in os.path.splitext(file)[0]):
                # spacing_len = （signal_len - im_size） //  train_num   报错无效字符，不能用小括号吗
                spacing_len = ss //  train_num
                choice = list(range(0, train_num*spacing_len, spacing_len))
                # choices = [i for i in range(data_DE.shape[0] - im_size)]  
                print(os.path.splitext(file)[0], "train")              
                # choice = random.sample(choices ,train_num)
                
                count=0
                for i in choice :
                # for i in range(im_sum):
                  start_number = i
                  # start_index, end_index = i * step, i * step + im_size
                  start_index, end_index = i , i + series
                  # print(start_index, end_index)
                  
                  sub_series = final_data[start_index:end_index,:].reshape(1,-1)
              
                  # print(sub_series)
                  # print(sub_series.shape)  #(1,128)
                  GAF_gasf = gasf.fit_transform(sub_series)
                  im = GAF_gasf[0]

                  im=im*255
                  
                  # filename = save_path + '%d.png' % i
                  # image.imsave(filename, im)
                  cv2.imwrite(save_path + '/train/' + os.path.splitext(file)[0] + '_' + str(i) + '-G' + '.png', im)
                  count+=1
                print(f'Total number of train_images numbers : {count}')  # 输出总数
                path='/content/12kDriveEnd_img/train'
                counter = 0
                for root,dirs,files in os.walk(path):    #遍历统计
                   for each in files:
                     counter += 1   #统计文件夹下文件个数
                print(f'train images number in folder:{counter}')               #输出结果
          elif ("_1") in os.path.splitext(file)[0]:
                spacing_len = ss //  val_num 
                choice = list(range(0, val_num*spacing_len, spacing_len))
                # choices = [i for i in range(data_DE.shape[0] - im_size)]
                print(os.path.splitext(file)[0], "val")
                # choice = random.sample(choices, val_num)
                count=0
                for i in choice:
                  start_number = i
                  start_index, end_index = i , i + series
                  
                  
                  sub_series = final_data[start_index:end_index,:].reshape(1,-1)
                  GAF_gasf = gasf.fit_transform(sub_series)
                  im = GAF_gasf[0]
                  im=im*255
                  cv2.imwrite(save_path + '/val/' + os.path.splitext(file)[0] + '_' + str(i) + '-G' + '.png', im)
                  count+=1
                print(f'Total number of val_images numbers : {count}')  # 输出总数
                
          elif ("_2") in os.path.splitext(file)[0]:
                spacing_len = ss //  test_num 
                choice = list(range(0, test_num*spacing_len, spacing_len))
                # choices = [i for i in range(data_DE.shape[0] - im_size)]
                print(os.path.splitext(file)[0], "test")
                # choice = random.sample(choices, test_num)
                count=0
                for i in choice:
                  start_index, end_index = i , i + series
                  
                  # sub_series = final_data[:, start_index:end_index]
                  sub_series = final_data[start_index:end_index,:].reshape(1,-1)
                  # print(sub_series.shape)
                  GAF_gasf = gasf.fit_transform(sub_series)
                  im = GAF_gasf[0]
                  im=im*255
                  cv2.imwrite(save_path + '/test/' + os.path.splitext(file)[0] + '_' + str(i) + '-G' + '.png', im)
                  count+=1
                print(f'Total number of test_images numbers : {count}')  # 输出总数  
          else:
                print("wrong!", os.path.splitext(file)[0])

          print('Finished creating img files for ' + file)
    print('- - - - - - - - - - - - - - - - - - - -')
data_path = '/content/Motor_Bearing_Fault_Diagnosis/data/12kDriveEnd'
normal_path = '/content/Motor_Bearing_Fault_Diagnosis/data/Normal_Baseline_Data'
img_save_path = '/content/12kDriveEnd_img0.2'
train_number = round(20000/26)  # Real train number/26
val_number = round(2500/13)   # Real val number/13
test_number = round(2500/13)  # Real test number/13
# matrix_size =128 * 128

# folder2img(img_save_path, data_path, train_number, val_number, test_number, matrix_size)
folder2img(img_save_path, data_path, train_number, val_number, test_number)
import os

path='/content/12kDriveEnd_img0.2/train'
count = 0
for root,dirs,files in os.walk(path):    #遍历统计
      for each in files:
             count += 1   #统计文件夹下文件个数
print(count)              #输出结果


 51

X147_DE_time
122281
OR007@3_3 train
Total number of train_images numbers : 769
train images number in folder:0
Finished creating img files for OR007@3_3.mat

X159_DE_time
122281
OR007@12_2 test
Total number of test_images numbers : 192
Finished creating img files for OR007@12_2.mat

X158_DE_time
121991
OR007@12_1 val
Total number of val_images numbers : 192
Finished creating img files for OR007@12_1.mat

X211_DE_time
121846
IR021_2 test
Total number of test_images numbers : 192
Finished creating img files for IR021_2.mat

X197_DE_time
121846
OR014@6_0 train
Total number of train_images numbers : 769
train images number in folder:0
Finished creating img files for OR014@6_0.mat

X169_DE_time
121846
IR014_0 train
Total number of train_images numbers : 769
train images number in folder:0
Finished creating img files for IR014_0.mat

X130_DE_time
121991
OR007@6_0 train
Total number of train_images numbers : 769
train images number in folder:0
Finished creating img files for OR007@6_0.m

##MAR preprocessing with Equal difference

In [5]:
import scipy.io
import random
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import image
from pyts.image import GramianAngularField
import time
import math
from pyts.image import MarkovTransitionField
# def folder2img(save_path, folder_path, train_num, val_num, test_num, img_size):
def folder2img(save_path, folder_path, train_num, val_num, test_num):
    files = os.listdir(folder_path)
    print(f'\n{len(files)}')
    if not os.path.exists(save_path + '/train'):
        os.makedirs(save_path + '/train')
    if not os.path.exists(save_path + '/val'):
        os.makedirs(save_path + '/val')
    if not os.path.exists(save_path + '/test'):
        os.makedirs(save_path + '/test')
    for file in files:
        
     if os.path.splitext(file)[1] == '.mat':
          file_path = folder_path + "/" + file
          raw_data = scipy.io.loadmat(file_path)
          for name in raw_data.keys():
              if "DE" in name:
                  column_DE = name
                  print(f'\n{column_DE}')
          data_DE = raw_data[column_DE]
          
          n_sample,_, = data_DE.shape
          print(n_sample)
          final_data=data_DE
            
          # im_size = 128
          series = 1000
          # step = 200  # step of slide window
          ss = n_sample - series
          # im_sum = (n_sample - im_size) // step
          # gasf = GramianAngularField(image_size=im_size, method='difference')
          mtf = MarkovTransitionField(image_size=0.2, n_bins = 8, strategy = 'quantile')
          if ("_3" in os.path.splitext(file)[0]) or ("_0" in os.path.splitext(file)[0]):
                spacing_len = ss //  train_num 
                choice = list(range(0, train_num*spacing_len, spacing_len))
                # choices = [i for i in range(data_DE.shape[0] - im_size)]
                print(os.path.splitext(file)[0], "train")
                # choice = random.sample(choices ,train_num)
                
                count=0
                for i in choice :
                # for i in range(im_sum):
                  
                  # window_size =10
                  # start_index, end_index = i  , i + im_size * window_size
                  start_index, end_index = i , i + series
                  # print(start_index, end_index)
                  
                  sub_series = final_data[start_index:end_index,:].reshape(1,-1)
                  # paa = PiecewiseAggregateApproximation(window_size=window_size)
                  # X_paa = paa.transform(sub_series)
              
                  # print(sub_series)
                  # print(sub_series.shape)  #(1,128)
                  # GAF_gasf = gasf.fit_transform(sub_series)
                  X_mtf = mtf.fit_transform(sub_series)
                  im = X_mtf[0]
                  # plt.imshow(X_mtf[0], cmap='rainbow', origin='lower')
                  # plt.colorbar(fraction=0.0457, pad=0.04)
                  im=im*255
                  
                  # filename = save_path + '%d.png' % i
                  # image.imsave(filename, im)
                  cv2.imwrite(save_path + '/train/' + os.path.splitext(file)[0] + '_' + str(i) + '-M' + '.png', im)
                  count+=1
                print(f'Total number of train_images  : {count}')  # 输出总数
                path='/content/12kDriveEnd_img0.2/train'
                counter = 0
                for root,dirs,files in os.walk(path):    #遍历统计
                   for each in files:
                     counter += 1   #统计文件夹下文件个数
                print(f'train images number in folder:{counter}')               #输出结果
          elif ("_1") in os.path.splitext(file)[0]:
                spacing_len = ss //  val_num 
                choice = list(range(0, val_num*spacing_len, spacing_len))
                # choices = [i for i in range(data_DE.shape[0] - im_size)]
                print(os.path.splitext(file)[0], "val")
                # choice = random.sample(choices, val_num)
                count=0
                for i in choice:
                  
                  # window_size =10
                  # start_index, end_index = i  , i + im_size * window_size
                  start_index, end_index = i , i + series
                  sub_series = final_data[start_index:end_index,:].reshape(1,-1)
                  # paa = PiecewiseAggregateApproximation(window_size=window_size)
                  # X_paa = paa.transform(sub_series)
                  
                  
                  # sub_series = final_data[start_index:end_index,:].reshape(1,-1)
                  # GAF_gasf = gasf.fit_transform(sub_series)
                  X_mtf = mtf.fit_transform(sub_series)
                  im = X_mtf[0]
                  im=im*255
                  cv2.imwrite(save_path + '/val/' + os.path.splitext(file)[0] + '_' + str(i) + '-M' + '.png', im)
                  count+=1
                print(f'Total number of val_images  : {count}')  # 输出总数
                
          elif ("_2") in os.path.splitext(file)[0]:
                spacing_len = ss //  test_num 
                choice = list(range(0, test_num*spacing_len, spacing_len))
                # choices = [i for i in range(data_DE.shape[0] - im_size)]
                print(os.path.splitext(file)[0], "test")
                # choice = random.sample(choices, test_num)
                count=0
                for i in choice:
                  # window_size =10
                  # start_index, end_index = i  , i + im_size * window_size
                  start_index, end_index = i , i + series
                  
                  # sub_series = final_data[:, start_index:end_index]
                  sub_series = final_data[start_index:end_index,:].reshape(1,-1)
                  # paa = PiecewiseAggregateApproximation(window_size=window_size)
                  # X_paa = paa.transform(sub_series)
                  # print(sub_series.shape)
                  # GAF_gasf = gasf.fit_transform(sub_series)
                  X_mtf = mtf.fit_transform(sub_series)
                  im = X_mtf[0]
                  im=im*255
                  cv2.imwrite(save_path + '/test/' + os.path.splitext(file)[0] + '_' + str(i) + '-M' + '.png', im)
                  count+=1
                print(f'Total number of test_images  : {count}')  # 输出总数  
          else:
                print("wrong!", os.path.splitext(file)[0])

          print('Finished creating img files for ' + file)
    print('- - - - - - - - - - - - - - - - - - - -')
data_path = '/content/Motor_Bearing_Fault_Diagnosis/data/12kDriveEnd'
normal_path = '/content/Motor_Bearing_Fault_Diagnosis/data/Normal_Baseline_Data'
img_save_path = '/content/12kDriveEnd_img0.2'
train_number = round(20000/26)  # Real train number/26
val_number = round(2500/13)  # Real val number/13
test_number = round(2500/13)  # Real test number/13
# matrix_size =128 * 128

# folder2img(img_save_path, data_path, train_number, val_number, test_number, matrix_size)
folder2img(img_save_path, data_path, train_number, val_number, test_number)
import os

path='/content/12kDriveEnd_img0.2/train'
count = 0
for root,dirs,files in os.walk(path):    #遍历统计
      for each in files:
             count += 1   #统计文件夹下文件个数
print(count)              #输出结果


51

X147_DE_time
122281
OR007@3_3 train
Total number of train_images  : 769
train images number in folder:19994
Finished creating img files for OR007@3_3.mat

X159_DE_time
122281
OR007@12_2 test
Total number of test_images  : 192
Finished creating img files for OR007@12_2.mat

X158_DE_time
121991
OR007@12_1 val
Total number of val_images  : 192
Finished creating img files for OR007@12_1.mat

X211_DE_time
121846
IR021_2 test
Total number of test_images  : 192
Finished creating img files for IR021_2.mat

X197_DE_time
121846
OR014@6_0 train
Total number of train_images  : 769
train images number in folder:20763
Finished creating img files for OR014@6_0.mat

X169_DE_time
121846
IR014_0 train
Total number of train_images  : 769
train images number in folder:21532
Finished creating img files for IR014_0.mat

X130_DE_time
121991
OR007@6_0 train
Total number of train_images  : 769
train images number in folder:22301
Finished creating img files for OR007@6_0.mat

X234_DE_time
122426
OR021@6_0 

#Normal signals transformed into imgs

##GAF

In [6]:
import scipy.io
import random
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import image
from pyts.image import GramianAngularField
import time
import math
# def folder2img(save_path, folder_path, train_num, val_num, test_num, img_size):
def folder2img(save_path, folder_path, train_num, val_num, test_num):
    files = os.listdir(folder_path)
    print(f'\n {len(files)}')
    if not os.path.exists(save_path + '/train'):
        os.makedirs(save_path + '/train')
    if not os.path.exists(save_path + '/val'):
        os.makedirs(save_path + '/val')
    if not os.path.exists(save_path + '/test'):
        os.makedirs(save_path + '/test')
    for file in files:
        
     if os.path.splitext(file)[1] == '.mat':
          file_path = folder_path + "/" + file
          raw_data = scipy.io.loadmat(file_path)
          for name in raw_data.keys():
              if "DE" in name:
                  column_DE = name
                  print(f'\n{column_DE}')
          data_DE = raw_data[column_DE]
          
          n_sample,_, = data_DE.shape
          print(n_sample)
          final_data=data_DE
            
          # im_size = 128

          series = 1000   # each time series

          # im_size = 200
          # step = 1000  # step of slide window
          
          ss = n_sample - series
          # im_sum = (n_sample - im_size) // step
          gasf = GramianAngularField(image_size=0.2, method='difference')
          if ("_3" in os.path.splitext(file)[0]) or ("_0" in os.path.splitext(file)[0]):
                # spacing_len = （signal_len - im_size） //  train_num   报错无效字符，不能用小括号吗
                spacing_len = ss //  train_num
                choice = list(range(0, train_num*spacing_len, spacing_len))
                # choices = [i for i in range(data_DE.shape[0] - im_size)]  
                print(os.path.splitext(file)[0], "train")              
                # choice = random.sample(choices ,train_num)
                
                count=0
                for i in choice :
                # for i in range(im_sum):
                  start_number = i
                  # start_index, end_index = i * step, i * step + im_size
                  start_index, end_index = i , i + series
                  # print(start_index, end_index)
                  
                  sub_series = final_data[start_index:end_index,:].reshape(1,-1)
              
                  # print(sub_series)
                  # print(sub_series.shape)  #(1,128)
                  GAF_gasf = gasf.fit_transform(sub_series)
                  im = GAF_gasf[0]
                  im=im*255
                  
                  # filename = save_path + '%d.png' % i
                  # image.imsave(filename, im)
                  cv2.imwrite(save_path + '/train/' + os.path.splitext(file)[0] + '_' + str(i) + '-G' + '.png', im)
                  count+=1
                print(f'Total number of train_images numbers : {count}')  # 输出总数
                path='/content/12kDriveEnd_img/train'
                counter = 0
                for root,dirs,files in os.walk(path):    #遍历统计
                   for each in files:
                     counter += 1   #统计文件夹下文件个数
                print(f'train images number in folder:{counter}')               #输出结果
          elif ("_1") in os.path.splitext(file)[0]:
                spacing_len = ss //  val_num 
                choice = list(range(0, val_num*spacing_len, spacing_len))
                # choices = [i for i in range(data_DE.shape[0] - im_size)]
                print(os.path.splitext(file)[0], "val")
                # choice = random.sample(choices, val_num)
                count=0
                for i in choice:
                  start_number = i
                  start_index, end_index = i , i + series
                  
                  
                  sub_series = final_data[start_index:end_index,:].reshape(1,-1)
                  GAF_gasf = gasf.fit_transform(sub_series)
                  im = GAF_gasf[0]
                  im=im*255
                  cv2.imwrite(save_path + '/val/' + os.path.splitext(file)[0] + '_' + str(i) + '-G' + '.png', im)
                  count+=1
                print(f'Total number of val_images numbers : {count}')  # 输出总数
                
          elif ("_2") in os.path.splitext(file)[0]:
                spacing_len = ss //  test_num 
                choice = list(range(0, test_num*spacing_len, spacing_len))
                # choices = [i for i in range(data_DE.shape[0] - im_size)]
                print(os.path.splitext(file)[0], "test")
                # choice = random.sample(choices, test_num)
                count=0
                for i in choice:
                  start_index, end_index = i , i + series
                  
                  # sub_series = final_data[:, start_index:end_index]
                  sub_series = final_data[start_index:end_index,:].reshape(1,-1)
                  # print(sub_series.shape)
                  GAF_gasf = gasf.fit_transform(sub_series)
                  im = GAF_gasf[0]
                  im=im*255
                  cv2.imwrite(save_path + '/test/' + os.path.splitext(file)[0] + '_' + str(i) + '-G' + '.png', im)
                  count+=1
                print(f'Total number of test_images numbers : {count}')  # 输出总数  
          else:
                print("wrong!", os.path.splitext(file)[0])

          print('Finished creating img files for ' + file)
    print('- - - - - - - - - - - - - - - - - - - -')
data_path = '/content/Motor_Bearing_Fault_Diagnosis/data/12kDriveEnd'
normal_path = '/content/Motor_Bearing_Fault_Diagnosis/data/Normal_Baseline_Data'
img_save_path = '/content/12kDriveEnd_img0.2'
train_number = round(20000/2/9)  # Real train number/2
val_number = round(2500/9)   # Real val number
test_number = round(2500/9)  # Real test number
# matrix_size =128 * 128

# folder2img(img_save_path, data_path, train_number, val_number, test_number, matrix_size)
folder2img(img_save_path, normal_path, train_number, val_number, test_number)
import os

path='/content/12kDriveEnd_img0.2/train'
count = 0
for root,dirs,files in os.walk(path):    #遍历统计
      for each in files:
             count += 1   #统计文件夹下文件个数
print(count)              #输出结果


 4

X097_DE_time
243938
NORMAL_0 train
Total number of train_images numbers : 1111
train images number in folder:0
Finished creating img files for NORMAL_0.mat

X099_DE_time
485063
NORMAL_2 test
Total number of test_images numbers : 278
Finished creating img files for NORMAL_2.mat

X098_DE_time
483903
NORMAL_1 val
Total number of val_images numbers : 278
Finished creating img files for NORMAL_1.mat

X100_DE_time
485643
NORMAL_3 train
Total number of train_images numbers : 1111
train images number in folder:0
Finished creating img files for NORMAL_3.mat
- - - - - - - - - - - - - - - - - - - -
40672


##MAR

In [7]:
import scipy.io
import random
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import image
from pyts.image import GramianAngularField
import time
import math
from pyts.image import MarkovTransitionField
# def folder2img(save_path, folder_path, train_num, val_num, test_num, img_size):
def folder2img(save_path, folder_path, train_num, val_num, test_num):
    files = os.listdir(folder_path)
    print(f'\n{len(files)}')
    if not os.path.exists(save_path + '/train'):
        os.makedirs(save_path + '/train')
    if not os.path.exists(save_path + '/val'):
        os.makedirs(save_path + '/val')
    if not os.path.exists(save_path + '/test'):
        os.makedirs(save_path + '/test')
    for file in files:
        
     if os.path.splitext(file)[1] == '.mat':
          file_path = folder_path + "/" + file
          raw_data = scipy.io.loadmat(file_path)
          for name in raw_data.keys():
              if "DE" in name:
                  column_DE = name
                  print(f'\n{column_DE}')
          data_DE = raw_data[column_DE]
          
          n_sample,_, = data_DE.shape
          print(n_sample)
          final_data=data_DE
            
          # im_size = 128
          series = 1000
          # step = 200  # step of slide window
          ss = n_sample - series
          # im_sum = (n_sample - im_size) // step
          # gasf = GramianAngularField(image_size=im_size, method='difference')
          mtf = MarkovTransitionField(image_size=0.2, n_bins = 10, strategy = 'quantile')
          if ("_3" in os.path.splitext(file)[0]) or ("_0" in os.path.splitext(file)[0]):
                spacing_len = ss //  train_num 
                choice = list(range(0, train_num*spacing_len, spacing_len))
                # choices = [i for i in range(data_DE.shape[0] - im_size)]
                print(os.path.splitext(file)[0], "train")
                # choice = random.sample(choices ,train_num)
                
                count=0
                for i in choice :
                # for i in range(im_sum):
                  
                  # window_size =10
                  # start_index, end_index = i  , i + im_size * window_size
                  start_index, end_index = i , i + series
                  # print(start_index, end_index)
                  
                  sub_series = final_data[start_index:end_index,:].reshape(1,-1)
                  # paa = PiecewiseAggregateApproximation(window_size=window_size)
                  # X_paa = paa.transform(sub_series)
              
                  # print(sub_series)
                  # print(sub_series.shape)  #(1,128)
                  # GAF_gasf = gasf.fit_transform(sub_series)
                  X_mtf = mtf.fit_transform(sub_series)
                  im = X_mtf[0]
                  im=im*255
                  
                  # filename = save_path + '%d.png' % i
                  # image.imsave(filename, im)
                  cv2.imwrite(save_path + '/train/' + os.path.splitext(file)[0] + '_' + str(i) + '-M' + '.png', im)
                  count+=1
                print(f'Total number of train_images numbers : {count}')  # 输出总数
                path='/content/12kDriveEnd_img/train'
                counter = 0
                for root,dirs,files in os.walk(path):    #遍历统计
                   for each in files:
                     counter += 1   #统计文件夹下文件个数
                print(f'train images number in folder:{counter}')               #输出结果
          elif ("_1") in os.path.splitext(file)[0]:
                spacing_len = ss //  val_num 
                choice = list(range(0, val_num*spacing_len, spacing_len))
                # choices = [i for i in range(data_DE.shape[0] - im_size)]
                print(os.path.splitext(file)[0], "val")
                # choice = random.sample(choices, val_num)
                count=0
                for i in choice:
                  
                  # window_size =10
                  # start_index, end_index = i  , i + im_size * window_size
                  start_index, end_index = i , i + series
                  sub_series = final_data[start_index:end_index,:].reshape(1,-1)
                  # paa = PiecewiseAggregateApproximation(window_size=window_size)
                  # X_paa = paa.transform(sub_series)
                  
                  
                  # sub_series = final_data[start_index:end_index,:].reshape(1,-1)
                  # GAF_gasf = gasf.fit_transform(sub_series)
                  X_mtf = mtf.fit_transform(sub_series)
                  im = X_mtf[0]
                  im=im*255
                  cv2.imwrite(save_path + '/val/' + os.path.splitext(file)[0] + '_' + str(i) + '-M' + '.png', im)
                  count+=1
                print(f'Total number of val_images numbers : {count}')  # 输出总数
                
          elif ("_2") in os.path.splitext(file)[0]:
                spacing_len = ss //  test_num 
                choice = list(range(0, test_num*spacing_len, spacing_len))
                # choices = [i for i in range(data_DE.shape[0] - im_size)]
                print(os.path.splitext(file)[0], "test")
                # choice = random.sample(choices, test_num)
                count=0
                for i in choice:
                  # window_size =10
                  # start_index, end_index = i  , i + im_size * window_size
                  start_index, end_index = i , i + series
                  
                  # sub_series = final_data[:, start_index:end_index]
                  sub_series = final_data[start_index:end_index,:].reshape(1,-1)
                  # paa = PiecewiseAggregateApproximation(window_size=window_size)
                  # X_paa = paa.transform(sub_series)
                  # print(sub_series.shape)
                  # GAF_gasf = gasf.fit_transform(sub_series)
                  X_mtf = mtf.fit_transform(sub_series)
                  im = X_mtf[0]
                  im=im*255
                  cv2.imwrite(save_path + '/test/' + os.path.splitext(file)[0] + '_' + str(i) + '-M' + '.png', im)
                  count+=1
                print(f'Total number of test_images numbers : {count}')  # 输出总数  
          else:
                print("wrong!", os.path.splitext(file)[0])

          print('Finished creating img files for ' + file)
    print('- - - - - - - - - - - - - - - - - - - -')
data_path = '/content/Motor_Bearing_Fault_Diagnosis/data/12kDriveEnd'
normal_path = '/content/Motor_Bearing_Fault_Diagnosis/data/Normal_Baseline_Data'
img_save_path = '/content/12kDriveEnd_img0.2'
train_number = round(20000/2/9)  # Real train number/2
val_number = round(2500/9)   # Real val number
test_number = round(2500/9)  # Real test number
# matrix_size =128 * 128

# folder2img(img_save_path, data_path, train_number, val_number, test_number, matrix_size)
folder2img(img_save_path, normal_path, train_number, val_number, test_number)
import os

path='/content/12kDriveEnd_img0.2/train'
count = 0
for root,dirs,files in os.walk(path):    #遍历统计
      for each in files:
             count += 1   #统计文件夹下文件个数
print(count)              #输出结果


4

X097_DE_time
243938
NORMAL_0 train
Total number of train_images numbers : 1111
train images number in folder:0
Finished creating img files for NORMAL_0.mat

X099_DE_time
485063
NORMAL_2 test
Total number of test_images numbers : 278
Finished creating img files for NORMAL_2.mat

X098_DE_time
483903
NORMAL_1 val
Total number of val_images numbers : 278
Finished creating img files for NORMAL_1.mat

X100_DE_time
485643
NORMAL_3 train
Total number of train_images numbers : 1111
train images number in folder:0
Finished creating img files for NORMAL_3.mat
- - - - - - - - - - - - - - - - - - - -
42894


## 获取原始文件路径list

In [11]:
import os
train_filePath = '/content/12kDriveEnd_img0.2/train'
train_file_path_list_ori = os.listdir(train_filePath)
val_filePath = '/content/12kDriveEnd_img0.2/val'
val_file_path_list_ori = os.listdir(val_filePath)
test_filePath = '/content/12kDriveEnd_img0.2/test'
test_file_path_list_ori = os.listdir(test_filePath)

# file_path_list_ori[:5]

## 转化为文件名list

In [12]:
train_file_name_list = list(set([file_path.split('-')[0] for file_path in train_file_path_list_ori]))
val_file_name_list = list(set([file_path.split('-')[0] for file_path in val_file_path_list_ori]))
test_file_name_list = list(set([file_path.split('-')[0] for file_path in test_file_path_list_ori]))
# file_name_list[:5],len(file_name_list)
# file_name_list,len(file_name_list)

## 构建名称与label id的映射关系

In [13]:
def get_label_num(file_name):
  if 'B007' in file_name:
    return 3
  elif 'B014' in file_name:
    return 6
  elif 'B021' in file_name:
    return 9
  elif 'OR007' in file_name:
    return 1
  elif 'OR014' in file_name:
    return 4
  elif 'OR021' in file_name:
    return 7
  elif 'IR007' in file_name:
    return 2
  elif 'IR014' in file_name:
    return 5
  elif 'IR021' in file_name:
    return 8
  else :
    return 0
get_label_num('OR1007@12_3_60568')

0

## 构建dataset类

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

In [ ]:
img_transform = transforms.Compose([
        transforms.Resize(200),
        # transforms.RandomHorizontalFlip(),
        transforms.ToTensor()])

In [15]:
import os
import pandas as pd
from torchvision.io import read_image
from PIL import Image

train_img_dir = '/content/12kDriveEnd_img0.2/train'
val_img_dir = '/content/12kDriveEnd_img0.2/val'
test_img_dir = '/content/12kDriveEnd_img0.2/test'
class CustomImageDataset(Dataset):
    # def __init__(self, file_name_list, img_dir,img_transform):
    def __init__(self, file_name_list, img_dir):
        self.file_name_list = file_name_list
        self.img_dir = img_dir
        # self.img_transform = img_transform

    def __len__(self):
        return len(self.file_name_list)

    def __getitem__(self, idx):
        file_name = self.file_name_list[idx]
        img_path_m = os.path.join(self.img_dir, self.file_name_list[idx]+'-M.png')
        img_path_g = os.path.join(self.img_dir, self.file_name_list[idx]+'-G.png')
        image_m = read_image(img_path_m)
        image_g = read_image(img_path_m)
        # image_m = Image.open(img_path_m)
        # image_g = Image.open(img_path_g)
        # image_tensor_m = self.img_transform(image_m)
        # image_tensor_g = self.img_transform(image_g)
        # label_name = file_name.split('_')[0]
        # label_id = le.transform([label_name])[0]
        label_id = get_label_num(file_name)

        return (image_m,image_g), label_id

# train_img_dataset = CustomImageDataset( file_name_list, train_img_dir,img_transform)
# val_img_dataset = CustomImageDataset( file_name_list, val_img_dir,img_transform)
# test_img_dataset = CustomImageDataset( file_name_list, test_img_dir,img_transform)
train_img_dataset = CustomImageDataset( train_file_name_list, train_img_dir)
val_img_dataset = CustomImageDataset( val_file_name_list, val_img_dir)
test_img_dataset = CustomImageDataset( test_file_name_list, test_img_dir)
train_img_dataset[2]

((tensor([[[40, 36, 31,  ..., 40, 36, 37],
           [38, 38, 37,  ..., 37, 36, 36],
           [37, 37, 37,  ..., 36, 34, 35],
           ...,
           [39, 37, 33,  ..., 39, 37, 37],
           [37, 35, 33,  ..., 37, 36, 36],
           [38, 36, 34,  ..., 38, 37, 36]]], dtype=torch.uint8),
  tensor([[[40, 36, 31,  ..., 40, 36, 37],
           [38, 38, 37,  ..., 37, 36, 36],
           [37, 37, 37,  ..., 36, 34, 35],
           ...,
           [39, 37, 33,  ..., 39, 37, 37],
           [37, 35, 33,  ..., 37, 36, 36],
           [38, 36, 34,  ..., 38, 37, 36]]], dtype=torch.uint8)),
 1)

## 构建dataloader

In [16]:
train_dataloader = DataLoader(train_img_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_img_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_img_dataset, batch_size=64, shuffle=True)

In [17]:
len(train_dataloader.dataset)

21447

# 模型

## base model

In [18]:
import torch.nn as nn
import torch.nn.functional as F

class Base_model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(35344, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 50)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

base_model = Base_model()
base_model(images[0]).shape

NameError: ignored

## CRNN模型

In [17]:
import torch.nn as nn
import torch.nn.functional as F
import torch

class BidirectionalLSTM(nn.Module):
    # Inputs hidden units Out
    def __init__(self, nIn, nHidden, nOut):
        super(BidirectionalLSTM, self).__init__()

        self.rnn = nn.LSTM(nIn, nHidden, bidirectional=True)
        self.embedding = nn.Linear(nHidden * 2, nOut)

    def forward(self, input):
        recurrent, _ = self.rnn(input)
        T, b, h = recurrent.size()
        t_rec = recurrent.view(T * b, h)

        output = self.embedding(t_rec)  # [T * b, nOut]
        output = output.view(T, b, -1)

        return output


class CRNN(nn.Module):
    # def __init__(self, imgH, nc, nclass, nh, n_rnn=2, leakyRelu=False, phi=0):
    def __init__(self, imgH, nc, nclass, nh, n_rnn=2, leakyRelu=False):
        super(CRNN, self).__init__()
        # assert imgH % 16 == 0, 'imgH has to be a multiple of 16'

        ks = [3, 3, 3, 3, 3, 3, 3]
        ps = [1, 1, 1, 1, 1, 1, 0]
        ss = [1, 1, 1, 1, 1, 1, (3, 1)]
        nm = [64, 128, 256, 256, 512, 512, 512]

        cnn = nn.Sequential()

        # if phi>=1 and phi<=3:
        #   self

        def convRelu(i, batchNormalization=False):
            nIn = nc if i == 0 else nm[i - 1]
            nOut = nm[i]
            cnn.add_module('conv{0}'.format(i),
                           nn.Conv2d(nIn, nOut, ks[i], ss[i], ps[i]))
            if batchNormalization:
                cnn.add_module('batchnorm{0}'.format(i), nn.BatchNorm2d(nOut))
            if leakyRelu:
                cnn.add_module('relu{0}'.format(i),
                               nn.LeakyReLU(0.2, inplace=True))
            else:
                cnn.add_module('relu{0}'.format(i), nn.ReLU(True))

        convRelu(0)
        cnn.add_module('pooling{0}'.format(0), nn.MaxPool2d(2, 2))  # 64*100*100
        convRelu(1)
        cnn.add_module('pooling{0}'.format(1), nn.MaxPool2d(2, 2))  # 128*50*50
        convRelu(2, True)
        convRelu(3)
        cnn.add_module('pooling{0}'.format(2), nn.MaxPool2d(2, 2))  # 256x25x25
        convRelu(4, True)
        convRelu(5)

        # cnn.add_module('pooling{0}'.format(3), nn.MaxPool2d(2, 2))  # 512*8*11

        cnn.add_module('pooling{0}'.format(3),
                       nn.MaxPool2d((3, 3), (3, 2), (1, 0)))  # 512*20*20

        convRelu(6)  # 512*2*10
        # self.Relu6 = convRelu(6)

        cnn.add_module('pooling{0}'.format(4),
                       nn.MaxPool2d((2, 3), (3, 1), (0, 1)))  # 512*1*10

        # #                            # 512*6*6=18432

        # self.flatten = nn.Flatten()    #32*18432

        # self.fc0 = nn.Linear(18432, 2048)
        # self.Relu7 = nn.ReLU(inplace=True)
        # self.drop0 = nn.Dropout(p=0.5)
        # self.fc1 = nn.Linear(2048, 2048)
        # self.Relu8 = nn.ReLU(inplace=True)

        # self.to_lstm = nn.Sequential(

        #     nn.ReLU(inplace=true),
        #     nn.Linear(2048, 10)
        # )

        self.cnn = cnn
        self.rnn = nn.Sequential(
            BidirectionalLSTM(512, nh, nh),
            BidirectionalLSTM(nh, nh, nclass))
        self.flatten = nn.Flatten()  # 32*100

        self.fc0 = nn.Linear(100, 10)
        self.Relu7 = nn.ReLU(inplace=True)
        # self.drop0 = nn.Dropout(p=0.5)
        self.fc1 = nn.Linear(2048, 10)
        # self.Relu8 = nn.ReLU(inplace=True)
        # self.softmax = nn.Softmax(dim=0)

    def forward(self, input):

        # conv features
        conv = self.cnn(input)  # [32, 512, 1, 10]
        b, c, h, w = conv.size()

        # print('conv:', conv.size())

        assert h == 1, "the height of conv must be 1"
        conv = conv.squeeze(2)  # b *512 * width
        conv = conv.permute(2, 0, 1)  # [w, b, c]

        rnn_out = self.rnn(conv)  # [10, 32, 4]   new[10, 32, 10]
        # print('rnn_out:', rnn_out.size())

        rnn_out = rnn_out.permute(1, 0, 2)  # [b, w, c]
        y = self.flatten(rnn_out)  # [b, w*c]
        # print('flatten:', y.size())
        output = self.fc0(y)
        # print('fc0:', output.size())
     

        return output

model_crnn=CRNN(200,1,10,64)

# model_crnn(images[0]).shape

## 完整模型1

In [ ]:
torch.cat([images[0],images[0]]).shape

torch.Size([8, 1, 200, 200])

In [ ]:
class Final_model(nn.Module):
    def __init__(self):
      super().__init__()
      self.base_model = base_model
      self.fc1 = nn.Linear(100, 50)
      self.fc2 = nn.Linear(50, 10)

    def forward(self, img_m,img_g):
      tensor1 = self.base_model(img_m)
      tensor2 = self.base_model(img_g)
      mid_tensor = torch.cat([tensor1,tensor2],1)
      mid_tensor = self.fc1(mid_tensor)
      final_out = self.fc2(mid_tensor)
      return final_out
final_model_1 = Final_model()
final_model_1(images[0],images[1]).shape

torch.Size([4, 10])

In [ ]:
final_model_2(images[0],images[1]).is_cuda

NameError: ignored

## 完整模型2

In [ ]:
torch.cat([train_images[0],train_images[0]]).shape

NameError: ignored

In [18]:
class Final_model(nn.Module):
    def __init__(self):
      super().__init__()
      self.base_model = model_crnn
      self.fc1 = nn.Linear(20, 10)
      # self.fc2 = nn.Linear(100, 10)

    def forward(self, img_m,img_g):
      tensor1 = self.base_model(img_m)
      tensor2 = self.base_model(img_g)
      mid_tensor = torch.cat([tensor1,tensor2],1)
      # final_out = self.fc1(mid_tensor)
      final_out = self.fc1(mid_tensor)
      return final_out
final_model_2 = Final_model()
# final_model_2(images[0],images[1]).shape

## 分配优化器和损失函数

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(final_model_1.parameters(), lr=0.001, momentum=0.9)

NameError: ignored

## 构建训练for循环

In [ ]:
for epoch in range(20):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        (img_m,img_g), labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = final_model_1(img_m,img_g)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 10 == 9:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 10:.3f}')
            running_loss = 0.0

print('Finished Training')

##原来的训练

In [ ]:
import torch
from torch import nn
# from model import CRNN
# from model import BidirectionalLSTM
from torch.utils.data import DataLoader
# from create_dataset import MyDataset
from  torch.utils.tensorboard import SummaryWriter
from torch.optim.lr_scheduler import CosineAnnealingLR, ExponentialLR
from torch.optim.lr_scheduler import StepLR
from torch.optim.lr_scheduler import MultiStepLR
import os
import shutil
def main():
    if os.path.exists('./loss_log'):
        shutil.rmtree('./loss_log')
    writer = SummaryWriter('./loss_log')
    learning_rate = 3e-3
    # batch_size = 128
    epochs = 400

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    # model = CRNN(200,1,10,256).to(device)
    
    model = final_model_2.to(device)


    # train_data = MyDataset('./train.csv', '/content/12kDriveEnd_img/train')
    # eval_data = MyDataset('./val.csv', '/content/12kDriveEnd_img/val')
    # test_data = MyDataset('./test.csv', '/content/12kDriveEnd_img/test')
    # train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    # eval_dataloader = DataLoader(eval_data, batch_size=batch_size, shuffle=True)
    # test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

    loss_fn = nn.CrossEntropyLoss()
    base_optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    scheduler = CosineAnnealingLR(base_optimizer, T_max=40, eta_min=5 * 1e-6)
    scheduler_1 = ExponentialLR(base_optimizer, gamma=0.95)
    scheduler_2 = StepLR(base_optimizer, step_size=3, gamma=0.9)
    scheduler_3 = MultiStepLR(
    optimizer=base_optimizer,
    milestones=[12, 50, 130],  # 设定调整的间隔数
    gamma=0.1,  # 系数
    last_epoch=-1
    )
#学习率的问题，loss那么大，一直不提升
    def model_test(epoch, dataloader, test_model, highest_score, is_train=True):
        size = len(dataloader.dataset)   #test总的文件名个数4174
        if not is_train:
          test_model.val()
        test_loss, correct = 0, 0
        total = 0
        with torch.no_grad():
            # for X, y in dataloader:
            for i, data in enumerate(train_dataloader, 0):  #i表示第i个batch_size
                (img_m,img_g), labels = data
              
                img_m  = img_m.float().to(device)
                img_g  = img_g.float().to(device)
                
                outputs = test_model(img_m,img_g)
                # the class with the highest energy is what we choose as prediction
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)  #labels.size(0)：分类数10
                correct += (predicted == labels.to(device)).sum().item()

                test_loss += loss_fn(outputs, labels.to(device)).item()
              
                # correct += (outputs.argmax(1) == labels.to(device)).type(torch.int).sum().item()
                # correct = correct/2
        test_loss /= size
        # correct /= size
        correct /= total
        highest_score = max(highest_score, correct)
        # print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')
        # print(f"\n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
        print(f"\n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
        niter = epoch * len(train_dataloader)
        writer.add_scalars('Train',
                           {'val_correct': correct}, niter)
        return highest_score

    def train_loop(epoch, dataloader, train_model, loss_function, optimizer):
        print('learning_rate: ', optimizer.param_groups[0]['lr'])
        size = len(dataloader.dataset)

        
        for i, data in enumerate(train_dataloader, 0):  
          # get the inputs; data is a list of [inputs, labels]
          (img_m,img_g), labels = data

          # zero the parameter gradients
          optimizer.zero_grad()

          # forward + backward + optimize
          img_m  = img_m.float().to(device)
          img_g  = img_g.float().to(device)
          outputs = final_model_2(img_m,img_g)
          loss = loss_function(outputs, labels.to(device))
          loss.backward()
          optimizer.step()
  
          # print statistics
          # running_loss += loss.item()
          if i % 100 == 0:    # i=batch=第几个batch_size
                loss, current = loss.item(), i * len(img_g)
                print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
                niter = epoch * len(train_dataloader) + i
                writer.add_scalars('Train',
                                   {'train_loss': loss}, niter)
                writer.add_scalars('Train',
                                   {'learning_rate': optimizer.param_groups[0]['lr']}, niter)
            # print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 10:.3f}')
            

    val_best_correct = 0
    val_before_correct = 0
    test_best_correct = 0
    test_before_correct = 0
    for t in range(epochs):
        print(f"Epoch {t + 1}\n-------------------------------")
        train_loop(t, train_dataloader, model, loss_fn, base_optimizer)
        val_best_correct = model_test(t, val_dataloader, model, val_best_correct)
        test_best_correct = model_test(t, test_dataloader, model, test_best_correct)
        scheduler_3.step()
        if val_best_correct > 0.090 and val_best_correct > val_before_correct:
            torch.save(model, './val_model/model_'+str(val_best_correct)+'.pth')
            print('save_model: ', val_best_correct)

        if test_best_correct > 0.090 and test_best_correct > test_before_correct:
            torch.save(model, './test_model/model_'+str(test_best_correct)+'.pth')
            print('save_model: ', test_best_correct)
        val_before_correct = val_best_correct
        test_before_correct = test_best_correct
        print('done, val_best_score: ', val_best_correct)
        print('done, test_best_score: ', test_best_correct)


if __name__ == '__main__':
    main()

Epoch 1
-------------------------------
learning_rate:  0.003
loss: 2.301127  [    0/21447]
loss: 2.282361  [ 6400/21447]
loss: 2.269553  [12800/21447]
loss: 2.273313  [19200/21447]

 Accuracy: 21.5%, Avg loss: 0.272103 


 Accuracy: 21.5%, Avg loss: 0.272065 

save_model:  0.2151349839138341
save_model:  0.2151349839138341
done, val_best_score:  0.2151349839138341
done, test_best_score:  0.2151349839138341
Epoch 2
-------------------------------
learning_rate:  0.003
loss: 2.229097  [    0/21447]
loss: 2.219172  [ 6400/21447]


In [ ]:
-------------------------------------------------------------------------------------

#修剪

In [23]:
import torch
import torch.nn.utils.prune as prune
import torchvision
import random
import sys
import torch_pruning
sys.path.append("/content/Model-Compression")

from model import *

device = 'cuda' if torch.cuda.is_available() else 'cpu'
random.seed(777)  #每次生成的随机数一样
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed(777)

In [ ]:
learning_rate = 1e-4
batch_size = 64
epoch = 20

In [ ]:
root = "/content/12kDriveEnd_img0.2/train"
transform = torchvision.transforms.ToTensor()
data_train = torchvision.datasets.CIFAR100(root, train = True, transform = transform, download = True)
data_test = torchvision.datasets.CIFAR100(root, train = False, transform = transform, download = True)

  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting ./CIFAR100/cifar-100-python.tar.gz to ./CIFAR100
Files already downloaded and verified


In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=data_train, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = torch.utils.data.DataLoader(dataset=data_test, batch_size=batch_size, shuffle=False, drop_last=True)

In [ ]:
model = ResNet(Bottleneck, [3, 4, 6, 3], num_classes=100).to(device)
checkpoint = torch.load("/content/Model-Compression/model.pth")
model.load_state_dict(checkpoint)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

Code for Training and Eval

In [ ]:
def train_one_epoch(print_result = False):
    model.train()
    loss_sum = 0.0
    accuracy_sum = 0.0
    length = 0

    for X, Y in train_loader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()

        pred = model(X)
        loss = criterion(pred, Y)
        pred_idx = torch.argmax(pred, 1)
        loss_sum += loss.item()
        accuracy_sum += torch.sum((pred_idx == Y).float()).item()
        length += X.size(0)

        loss.backward()
        optimizer.step()

    if(print_result):
        print("loss :", loss_sum / length)
        print("accuracy:", accuracy_sum / length)

In [ ]:
def eval():
    with torch.no_grad():
        model.eval()
        loss_sum = 0.0
        accuracy_sum = 0.0
        length = 0

        for X, Y in train_loader:
            X = X.to(device)
            Y = Y.to(device)

            pred = model(X)
            loss = criterion(pred, Y)
            pred_idx = torch.argmax(pred, 1)
            loss_sum += loss.item()
            accuracy_sum += torch.sum((pred_idx == Y).float()).item()
            length += X.size(0)
            
        print("loss :", loss_sum / length)
        print("accuracy:", accuracy_sum / length)

In [ ]:
def train():
    for i in range(epoch):
        print("EPOCH[" + str(i + 1) + "]")
        print("==== train ====")
        train_one_epoch(print_result=True)
        
        print("==== eval ====")
        eval()

##Code for Pruning

In [ ]:
""" code for pruning specific filters

DG = torch_pruning.DependencyGraph()
DG.build_dependency(model, example_inputs=torch.randn(1, 3, 32, 32).to("cuda"))
pruning_idxs = [0, 1, 2, 3, 4] # remove filter 0 ~ 4
pruning_group = DG.get_pruning_group(model.conv1, torch_pruning.prune_conv_out_channels, idxs = pruning_idxs)
print(pruning_group)
if DG.check_pruning_group(pruning_group):
    pruning_group.exec()
"""

' code for pruning specific filters\n\nDG = torch_pruning.DependencyGraph()\nDG.build_dependency(model, example_inputs=torch.randn(1, 3, 32, 32).to("cuda"))\npruning_idxs = [0, 1, 2, 3, 4] # remove filter 0 ~ 4\npruning_group = DG.get_pruning_group(model.conv1, torch_pruning.prune_conv_out_channels, idxs = pruning_idxs)\nprint(pruning_group)\nif DG.check_pruning_group(pruning_group):\n    pruning_group.exec()\n'

##自己的

In [24]:
# code for pruning whole model

# example_inputs = torch.randn(1, 3, 32, 32).to("cuda")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
example_inputs = torch.randn(64, 1, 200, 200)
model=final_model_2.to(device)
imp = torch_pruning.importance.MagnitudeImportance(p = 2)  # importance criterion for parameter selections
ignored_layers = []  # ignore some layers that should not be pruned, e.g., the final classifier layer.
for m in model.modules():
    if isinstance(m, torch.nn.Linear) and m.out_features == 100:
        ignored_layers.append(m)  # DO NOT prune the final classifier!

#ch_sparsity = 0.5 means 50% of the filters are pruned
pruner = torch_pruning.pruner.MagnitudePruner(
    model=model, 
    example_inputs=example_inputs, 
    importance=imp, # importance criterion for parameter selection
    iterative_steps=5, # You can prune your model to the target sparsity iteratively.
    ch_sparsity=0.7, # remove 50% channels, ResNet18 = {64, 128, 256, 512} => ResNet18_Half = {32, 64, 128, 256}
    ignored_layers=ignored_layers
    )

base_macs, base_nparams = torch_pruning.utils.count_ops_and_params(model=model, example_inputs=example_inputs)  #mac就是flops
print("basemacs :", base_macs, "basenparams :", base_nparams)
for i in range(5):  #5=iterative_steps

    pruner.step()  # the pruner.step will remove some channels from the model with least importance

    macs, nparams = torch_pruning.utils.count_ops_and_params(model=model, example_inputs=example_inputs)#  Do whatever you like here, such as fintuning
    print("iter [", i, "] macs :", macs, "nparams :", nparams)

    #finetune
    optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

    print("==== train ====")
    for j in range(10):
        train_one_epoch(print_result = True)
    
    print("==== eval ====")
    eval()

TypeError: ignored

In [25]:
# code for pruning whole model

example_inputs = torch.randn(64, 1, 200, 200).to("cuda")

imp = torch_pruning.importance.MagnitudeImportance(p = 2)  # importance criterion for parameter selections
ignored_layers = []  # ignore some layers that should not be pruned, e.g., the final classifier layer.
for m in model.modules():
    if isinstance(m, torch.nn.Linear) and m.out_features == 100:
        ignored_layers.append(m)  # DO NOT prune the final classifier!

#ch_sparsity = 0.5 means 50% of the filters are pruned
pruner = torch_pruning.pruner.MagnitudePruner(
    model=model, 
    example_inputs=example_inputs, 
    importance=imp, # importance criterion for parameter selection
    iterative_steps=5, # You can prune your model to the target sparsity iteratively.
    ch_sparsity=0.7, # remove 50% channels, ResNet18 = {64, 128, 256, 512} => ResNet18_Half = {32, 64, 128, 256}
    ignored_layers=ignored_layers
    )

base_macs, base_nparams = torch_pruning.utils.count_ops_and_params(model=model, example_inputs=example_inputs)
print("basemacs :", base_macs, "basenparams :", base_nparams)
for i in range(5):  #5=iterative_steps

    pruner.step()  # the pruner.step will remove some channels from the model with least importance

    macs, nparams = torch_pruning.utils.count_ops_and_params(model=model, example_inputs=example_inputs)#  Do whatever you like here, such as fintuning
    print("iter [", i, "] macs :", macs, "nparams :", nparams)

    #finetune
    optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

    print("==== train ====")
    for j in range(10):
        train_one_epoch(print_result = True)
    
    print("==== eval ====")
    eval()

TypeError: ignored

In [ ]:
eval()

In [ ]:
# torch.save(model, "/content/Model-Compression/pruned.pth")
# model = torch.load("/content/Model-Compression/pruned.pth")